# Домашнее задание третьего урока

Сначала пара задач на теорию, а потом продолжим тренироваться с данными мобильного приложения📱.

**installs** — содержит данные об установках приложения по дням.

* DeviceID — идентификатор устройства, на которое было установлено приложение;
* InstallationDate — дата установки приложения;
* InstallCost — цена установки приложения в рублях;
* Platform — платформа, на которой было установлено приложение (iOS/ Android);
* Source — источник установки приложения (магазин приложения/ рекламная система/ переход с сайта).

**events** — содержит данные о том, как активно пользователи просматривают товары в приложении по дням.

* DeviceID — идентификатор устройства, на котором используется приложение;
* AppPlatform — платформа, на которой используется приложение (iOS/ Android);
* EventDate — дата, за которую собрана статистика;
* events — количество просмотров всех товаров за этот день у этого DeviceID.

**checks** — содержит данные о покупках пользователей в приложении по дням

* UserID — идентификатор пользователя;
* Rub — суммарный чек пользователя на дату;
* BuyDate — дата, за которую собрана статистика.

Особенность приложения заключается в том, что для просмотра товаров не обязательно авторизовываться. До момента авторизации про пользователя известен только его DeviceID — идентификатор устройства. При этом для совершения покупки логин обязателен. На моменте авторизации пользователю присваивается UserID, и тогда мы уже знаем два его идентификатора: DeviceID (устройство) и UserID (логин). Так как на этапах установки приложения и просмотра каталога пользователь еще может быть не авторизован, там мы сохраняем только DeviceID. Но так как покупки нельзя совершить без авторизации, то покупки сохраняются только с UserID. Для того чтобы просмотры и установки можно было объединить с покупками, нам нужна таблица соответствия DeviceID к UserID, то есть таблица devices:

* DeviceID — идентификатор устройства;
* UserID — идентификатор пользователя.


**1. Сопоставьте основные виды JOIN с описанием:**

CROSS JOIN **производит декартово произведение таблиц целиком**

LEFT OUTER JOIN **возвращает всю левую таблицу A и соответствующие строки из правой таблицы B**

RIGHT OUTER JOIN **возвращает всю правую таблицу B и соответствующие строки из левой таблицы A**

INNER JOIN **возвращает только совпадающие строки**

FULL OUTER JOIN **возвращает не совпадающие и совпадающие строки из обеих таблиц**


**2. Соотнесите номера таблиц table 3 (1-4) с видом JOIN, который был применен при ее формировании:**

1.
![](https://ucarecdn.com/cbabccc8-119b-4330-9d94-a98a3850ca62/)
2.
![](https://ucarecdn.com/2cbc54f7-42e1-41a1-99d2-d346c247b751/)
3.
![](https://ucarecdn.com/4cfd3075-7c75-4816-a866-befc4dcf24f8/)
4.
![](https://ucarecdn.com/5ebb968c-1cb3-4b3d-93e7-fa2b8181aafa/)

* 4 **LEFT JOIN**
* 2 **CROSS JOIN**
* 3 **INNER JOIN**
* 1 **RIGHT JOIN**


**3. У пользователя может быть два идентификатора – UserID и DeviceID. В таблице checks есть только UserID, в остальных – только DeviceID. Во вспомогательной таблице devices есть и UserID, и DeviceID.**

**Давайте с помощью JOIN дополним таблицу events (left) данными о UserID пользователей из таблицы devices (right). Для некоторых DeviceID не будет пары UserID из таблицы devices – подумайте, какой вид JOIN подойдет, чтобы не потерять те строки, где DeviceID есть в events, но нет в devices.**

**Укажите UserID из первой строки результирующей таблицы, используя сортировку по убыванию по полю DeviceID.**
```sql
SELECT
    e.AppPlatform AS AppPlatform,    
    e.DeviceID AS DeviceID,    
    e.EventDate AS EventDate,    
    e.events AS events,    
    d.UserID AS UserID    
FROM 
    events as e    
LEFT JOIN devices AS d
    ON e.DeviceID = d.DeviceID    
ORDER BY 
    DeviceID DESC 
LIMIT 1
```
Ответ: `8158889386484182099`


**4. Давайте проверим, пользователи пришедшие из какого источника совершили наибольшее число покупок. В качестве ответа выберите название Source, юзеры которого совершили больше всего покупок.**

Hint: Для этого используйте UserID, DeviceID и Source из соответствующих таблиц. Считать уникальные значения здесь не нужно.
```sql
SELECT
    i.Source,     
    COUNT(c.UserID) as cnt    
FROM 
    checks as c    
JOIN devices as d
    ON c.UserID = d.UserID   
JOIN installs as i
    ON d.DeviceID = i.DeviceID   
GROUP BY 
    i.Source   
ORDER BY 
    cnt DESC   
LIMIT 1
```

* Source_5
* Source_26
* Source_9
* Source_15
* **Source_27**
* Source_14


**5. Теперь выясним, сколько всего уникальных юзеров что-то купили в нашем приложении.**

**Объедините нужные таблицы, посчитайте число уникальных UserID для каждого источника (Source), и в качестве ответа укажите число пользователей, пришедших из Source_7.**

Hint: checks – покупки, devices – соответствие, installs – информация об источнике.
```sql
SELECT 
    COUNT(DISTINCT c.UserID) as Uniq_users_count,    
    i.Source as Source    
FROM 
    checks as c   
JOIN devices as d
    ON c.UserID = d.UserID  
JOIN installs as i
    ON d.DeviceID = i.DeviceID    
WHERE i.Source = 'Source_7'
GROUP BY 
    i.Source   
LIMIT 1
```

Ответ: `11432`


**6. Самое время посмотреть на общую выручку, а также минимальный, максимальный и средний чек. Рассчитайте нужные показатели и соотнесите полученные значения:**
```sql
SELECT
    i.Source,     
    SUM(Rub) as Revenue,    
    MIN(Rub) as MinRub,   
    MAX(Rub) as MaxRub,    
    AVG(Rub) as AvgRub    
FROM 
    checks as c    
JOIN devices as d
    ON c.UserID = d.UserID    
JOIN installs as i
    ON d.DeviceID = i.DeviceID    
GROUP BY 
    i.Source    
LIMIT 50
```

* Общая сумма денег, полученная от пользователей Source_1 - **74**
* Общая выручка для Source_13 - **2752**
* Максимальная сумма заказа среди пользователей из Source_10 - **10**
* Средний чек пользователей Source_21 - **5.5555556**
* Максимальный чек пользователей из Source_14 - **38189**


**7. ведите идентификаторы устройств пользователей, которые совершили как минимум одну покупку за последний месяц (октябрь 2019). Используйте сортировку по возрастанию DeviceID и укажите минимальное значение.**

Hint: для извлечения месяца из даты можно использовать toMonth() или  toStartOfMonth(), предварительно приведя BuyDate к типу date.
```sql
SELECT *
FROM checks as c
JOIN devices as d
ON c.UserID=d.UserID
WHERE toStartOfMonth(CAST(BuyDate as date)) = '2019-10-01' 
ORDER BY DeviceID
LIMIT 10
```
Ответ: `22374478679519`


**8. Проверим, сколько товаров (events) в среднем просматривают пользователи с разных платформ (Platform), и пришедшие из разных источников  (Source). Для этого объедините таблицы events и installs, и посчитайте, сколько просмотров в среднем приходится на каждую пару платформа-канал привлечения.**

**Отсортируйте полученную табличку по убыванию среднего числа просмотров. В качестве ответа укажите платформу и источник, пользователи которого в среднем просматривали товары бóльшее число раз.**
```sql
SELECT
    e.AppPlatform,   
    i.Source,    
    AVG(e.events) as avg_events   
FROM
    events as e  
JOIN 
    installs as i    
ON 
    e.DeviceID = i.DeviceID    
GROUP BY
    e.AppPlatform,    
    i.Source   
ORDER BY 
    avg_events DESC    
LIMIT 10
```
Ответ: Платформа **IOS**, номер источника: **20**.


**9. Давайте посчитаем число уникальных DeviceID в инсталлах, для которых присутствуют просмотры в таблице events с разбивкой по платформам (поле Platform). Для этого можно отобрать все строки только из таблицы installs, для которых нашлось соответствие в таблице events.**

**В качестве ответа введите число инсталлов, для которых нашлись просмотры, на платформе android.**
```sql
SELECT 
    COUNT(DISTINCT(DeviceID)) AS devices_number,    
    Platform     
FROM 
    installs AS i     
LEFT SEMI JOIN 
    events AS ev    
ON 
    i.DeviceID=ev.DeviceID    
GROUP BY 
    Platform
```

Ответ: `9051220`


**10. Давайте теперь посчитаем конверсию из инсталла в просмотр с разбивкой по платформе инсталла – в данном случае это доля DeviceID, для которых есть просмотры, от всех DeviceID в инсталлах.** 

**Для этого нужно объединить таблицы installs и events так, чтобы получить все DeviceID инсталлов и соответствующие им DeviceID из events, посчитать число уникальных DeviceID инсталлов (1) и соответствующих DeviceID из events (2) и вычислить долю (2) от (1). В качестве ответа укажите значение конверсии из инсталла в просмотр на платформе ios.**

Внимание: ответ указать не в процентах, а именно в виде доли (т.е. не нужно домножать полученный ответ на 100). 
```sql
SELECT 
    Platform,     
    COUNT(DISTINCT(ev.DeviceID)) / COUNT(DISTINCT(i.DeviceID)) AS Conversion    
FROM 
    installs AS i     
LEFT JOIN
    events AS ev    
ON 
    i.DeviceID=ev.DeviceID     
GROUP BY 
    Platform
```

Ответ: `0.983182191473886`
    
**11. Представим, что в логирование DeviceID в событиях закралась ошибка - часть ID была записана в базу некорректно. Это привело к тому, что в таблице с событиями появились DeviceID, для которых нет инсталлов. Нам надо отобрать примеры DeviceID из таблицы event, которых нет в таблице installs, чтобы отправить их команде разработчиков на исправление.** 

**Выведите 10 уникальных DeviceID, которые присутствуют в таблице events, но отсутствуют в installs, отсортировав их в порядке убывания. В качестве ответа введите первый DeviceID из списка.**
```sql
SELECT 
    DISTINCT(DeviceID)    
FROM 
    events AS ev    
LEFT ANTI JOIN
    installs AS i    
ON 
    i.DeviceID=ev.DeviceID   
ORDER BY 
    DeviceID DESC    
LIMIT 10
```

Ответ: `18446741643567719656`